In [2]:
%%local
import os
import json
username = os.environ['JUPYTERHUB_USER']
namespace = os.environ['CI_NAMESPACE']
project = os.environ['CI_PROJECT']

configuration = dict(
    name = f"{username}-{namespace}-{project}",
    executorMemory = "64G",
    executorCores = 2,
    numExecutors = 2,
    conf = {
        # "spark.pyspark.python": "/opt/anaconda3/bin/python3", # Use python3
        "spark.jars.repositories": "https://repos.spark-packages.org"
    }
                
)

from IPython import get_ipython
ipython = get_ipython()
ipython.run_cell_magic('configure', line="-f",  cell=json.dumps(configuration))

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
7188,application_1618324153128_6903,pyspark,idle,Link,Link,None,
7192,application_1618324153128_6907,pyspark,idle,Link,Link,None,
7196,application_1618324153128_6922,pyspark,busy,Link,Link,None,
7197,application_1618324153128_6924,pyspark,idle,Link,Link,None,
7198,application_1618324153128_6927,pyspark,idle,Link,Link,None,
7199,application_1618324153128_6928,pyspark,idle,Link,Link,None,
7200,application_1618324153128_6929,pyspark,idle,Link,Link,None,


In [3]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
7201,application_1618324153128_6930,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Load Actual Data from SBB:

In [4]:
import pyspark.sql.functions as F

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#read the data from sbb
actual = spark.read.orc("/data/sbb/orc/istdaten")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
###################### Calculate the delay information
#Rename German column names to english
actual = actual.selectExpr("BETRIEBSTAG as date",
                           "FAHRT_BEZEICHNER as trip_id",
                           "BETREIBER_ID as operator_id",
                           "BETREIBER_ABK as operator_abk",
                           "BETREIBER_NAME as operator_name",
                           "PRODUKT_ID as transport_type",
                           "LINIEN_ID as train_number",
                           "LINIEN_TEXT as train_type1",
                           #"UMLAUF_ID as",
                           "VERKEHRSMITTEL_TEXT as train_type2",
                           "ZUSATZFAHRT_TF as additional_trip",
                           "FAELLT_AUS_TF as failed_trip",
                           "BPUIC as stop_id",
                           "HALTESTELLEN_NAME as stop_name",
                           "ANKUNFTSZEIT as schedule_arr_time",
                           "AN_PROGNOSE as actual_arr_time",
                           "AN_PROGNOSE_STATUS as arr_status",
                           "ABFAHRTSZEIT as schedule_dept_time",
                           "AB_PROGNOSE as actual_dept_time",
                           "AB_PROGNOSE_STATUS as dept_status",
                           "DURCHFAHRT_TF as does_not_stop"
                          )

#change the format of time
actual = actual.withColumn('schedule_arr_time', F.unix_timestamp('schedule_arr_time', 'dd.MM.yyy HH:mm'))
actual = actual.withColumn('actual_arr_time', F.unix_timestamp('actual_arr_time', 'dd.MM.yyy HH:mm'))
actual = actual.withColumn('schedule_dept_time', F.unix_timestamp('schedule_dept_time', 'dd.MM.yyy HH:mm'))
actual = actual.withColumn('actual_dept_time', F.unix_timestamp('actual_dept_time', 'dd.MM.yyy HH:mm'))
#compute the arrival and depature delay between the actual and schedule timestamp
actual = actual.withColumn("arr_delay", F.col('actual_arr_time')-F.col('schedule_arr_time'))
actual = actual.withColumn("dept_delay", F.col('actual_dept_time')-F.col('schedule_dept_time'))
#convert the format of time back
actual = actual.withColumn('schedule_arr_time', F.from_unixtime('schedule_arr_time', 'dd.MM.yyy HH:mm'))
actual = actual.withColumn('actual_arr_time', F.from_unixtime('actual_arr_time', 'dd.MM.yyy HH:mm'))
actual = actual.withColumn('schedule_dept_time', F.from_unixtime('schedule_dept_time', 'dd.MM.yyy HH:mm'))
actual = actual.withColumn('actual_dept_time', F.from_unixtime('actual_dept_time', 'dd.MM.yyy HH:mm'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
#read the file stop_nodes
stops = spark.read.parquet('/user/{0}/stops_node.parquet/*.parquet'.format('zhou'))
#filter the data in actual data and only keep stops within 15km from Zurich
actual_filter = actual.join(stops,['stop_id','stop_name'],'inner')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
actual_filter.select('stop_id').distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1459

In [9]:
#save the data for processed actual data
#actual_filter.write.parquet('/user/{0}/filtered_actual_data.parquet'.format("zhou"))
actual_filter = spark.read.parquet('/user/{0}/filtered_actual_data.parquet/*.parquet'.format('zhou'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
####Filter out data for weekend
### add a column of dayofweek
actual_filter = actual_filter.withColumn("dayofWeek", F.dayofweek(F.from_unixtime(F.unix_timestamp('date', 'dd.MM.yyy'))))
### filter out dayofWeek = 1 & 7(sunday and saturday)
actual_weekday = actual_filter.filter("dayofWeek >= 2 AND dayofWeek <= 6" )
## filter trips on the working hours of day (9am to 5pm)
actual_weekday_limited = actual_weekday.withColumn('arr_hour', F.hour(F.from_unixtime(F.unix_timestamp('actual_arr_time', 'dd.MM.yyy HH:mm'))))
actual_weekday_limited = actual_weekday_limited.withColumn('dept_hour', F.hour(F.from_unixtime(F.unix_timestamp('actual_dept_time', 'dd.MM.yyy HH:mm'))))
actual_weekday_limited = actual_weekday_limited.filter("arr_hour >= 9 AND arr_hour <= 17 AND dept_hour >= 9 AND dept_hour <= 17" )
actual_weekday_limited.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----------+--------------------+-----------+------------+--------------------+--------------+------------+-----------+-----------+---------------+-----------+-----------------+----------------+----------+------------------+----------------+-----------+-------------+---------+----------+---------+--------+-------------+--------------+-----------------+---------+--------+---------+
|stop_id|           stop_name|      date|             trip_id|operator_id|operator_abk|       operator_name|transport_type|train_number|train_type1|train_type2|additional_trip|failed_trip|schedule_arr_time| actual_arr_time|arr_status|schedule_dept_time|actual_dept_time|dept_status|does_not_stop|arr_delay|dept_delay| stop_lat|stop_lon|location_type|parent_station|dist_to_zurich_km|dayofWeek|arr_hour|dept_hour|
+-------+--------------------+----------+--------------------+-----------+------------+--------------------+--------------+------------+-----------+-----------+--------------

In [11]:
#filter out the data which have null values at schedule arr_time and actual arr_time
actual_weekday_limited = actual_weekday_limited.where((actual_weekday_limited.schedule_arr_time != ''))
actual_weekday_limited = actual_weekday_limited.where((actual_weekday_limited.actual_arr_time != ''))
#filter data where does_not_stop = True
actual_weekday_limited = actual_weekday_limited.filter("does_not_stop == 'false'")
actual_weekday_limited = actual_weekday_limited.filter("additional_trip == 'false'")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
#save the data for processed actual data
#actual_weekday_limited.write.mode("overwrite").parquet('/user/{0}/weekday_data.parquet'.format("zhou"))
#read data from parquet
df = spark.read.parquet('/user/{0}/weekday_data.parquet/*.parquet'.format('zhou'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Consider that the delay might vary based on morning, noon and afternoon of hour, so we need to group data based on hour

In [14]:
#add a column day_time in the dataframe
df = df.withColumn('day_time', F.when(df.arr_hour <= 11 , 'morning').when(df.arr_hour >= 14, 'afternoon').otherwise('noon'))
df = df.withColumn('minute_arr_delay', F.col('arr_delay')/60)
df = df.withColumn('minute_dept_delay', F.col('dept_delay')/60)
#set minute_arr_delay which are minus to 0
#df = df.withColumn('minute_arr_delay', F.when(df.minute_arr_delay >= 0, df.minute_arr_delay).otherwise(0))
df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------------+----------+--------------+-----------+------------+--------------------+--------------+------------+-----------+-----------+---------------+-----------+-----------------+----------------+----------+------------------+----------------+-----------+-------------+---------+----------+---------+--------+-------------+--------------+-----------------+---------+--------+---------+--------+----------------+-----------------+
|stop_id|       stop_name|      date|       trip_id|operator_id|operator_abk|       operator_name|transport_type|train_number|train_type1|train_type2|additional_trip|failed_trip|schedule_arr_time| actual_arr_time|arr_status|schedule_dept_time|actual_dept_time|dept_status|does_not_stop|arr_delay|dept_delay| stop_lat|stop_lon|location_type|parent_station|dist_to_zurich_km|dayofWeek|arr_hour|dept_hour|day_time|minute_arr_delay|minute_dept_delay|
+-------+----------------+----------+--------------+-----------+------------+--------------------+--------

In [15]:
group_stop_id = df.groupby(['stop_id','stop_name','transport_type','day_time']).agg(F.mean('minute_arr_delay').alias('mean_arr_delay'),F.stddev('minute_arr_delay').alias('std_arr_delay'),F.count('minute_arr_delay').alias('data_nb'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
group_stop_id.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+--------------+---------+-------------------+------------------+-------+
|stop_id|           stop_name|transport_type| day_time|     mean_arr_delay|     std_arr_delay|data_nb|
+-------+--------------------+--------------+---------+-------------------+------------------+-------+
|8591424|Zürich, Waserstrasse|           Bus|  morning|  1.346885672467068|2.3935700983986057|  36894|
|8576263|Zürich, Hungerber...|           Bus|afternoon|  1.427608904896451|2.8078954916353696|  64414|
|8591040|Zürich, Altenhofs...|           Bus|     noon| 0.9122556556116846|1.2659045194959875|   9106|
|8591438|Zürich, Witikon Z...|           Bus|afternoon| 3.6037155840684307|27.204862317478756|   7482|
|8590677|Kilchberg ZH, Hor...|           Bus|afternoon| 0.7299471132657558|1.3805475585085765|  18152|
|8503304|           Kemptthal|           Zug|     noon| 0.6043938161106591|0.9464229331124264|   6145|
|8502277|       Rudolfstetten|           Zug|  morning|-0.087044992086818

In [17]:
group_trip_id = df.groupby(['trip_id','transport_type','day_time']).agg(F.mean('minute_arr_delay').alias('mean_arr_delay'),F.stddev('minute_arr_delay').alias('std_arr_delay'),F.count('minute_arr_delay').alias('data_nb'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
group_trip_id.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------+---------+--------------------+------------------+-------+
|             trip_id|transport_type| day_time|      mean_arr_delay|     std_arr_delay|data_nb|
+--------------------+--------------+---------+--------------------+------------------+-------+
|       85:31:632:000|           Zug|  morning| 0.25734710087370927|1.0345364920883169|   7554|
|85:3849:57342-310...|          Tram|     noon| 0.24782187802516942|0.7047441665844718|   1033|
|85:3849:58260-470...|          Tram|afternoon|  0.1638888888888889|0.7525824722859792|    360|
|85:3849:58578-630...|          Tram|     noon|-0.01222826086956...|0.9395975007801722|    736|
|85:3849:58754-630...|          Tram|  morning|0.028532608695652172|0.7897307000028952|    736|
|85:3849:59283-480...|          Tram|afternoon|  0.2807971014492754|0.7895373559461352|    552|
|85:3849:60017-540...|          Tram|afternoon|  1.5615384615384615|1.9726967578696029|    130|
|85:3849:60399-230...|          Tram|aft

In [19]:
group_transport_id = df.groupby(['transport_type','day_time']).agg(F.mean('minute_arr_delay').alias('mean_arr_delay'),F.stddev('minute_arr_delay').alias('std_arr_delay'),F.count('minute_arr_delay').alias('data_nb'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
group_transport_id = group_transport_id.where((group_transport_id.transport_type != ''))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
group_transport_id.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+---------+-------------------+------------------+--------+
|transport_type| day_time|     mean_arr_delay|     std_arr_delay| data_nb|
+--------------+---------+-------------------+------------------+--------+
|           Bus|afternoon| 1.1462871854944685| 4.605782042108002|30737019|
|           Bus|     noon| 0.9182333681342736| 3.573711127040804|13780279|
|           Zug|     noon| 0.2842532960848372|1.1461260597005187| 1056329|
|           Bus|  morning|  0.897890218436548|3.6198311308265163|20460175|
|          Tram|afternoon| 0.5156668103307795| 2.633061853134485|14236912|
|           Zug|afternoon| 0.5429038774016444|1.3673454147850397| 2217387|
|           Zug|  morning|0.37131289799330364|1.2884397112035477| 1586239|
|          Tram|     noon|  0.437595961601139|2.4570991234560315| 6961769|
|          Tram|  morning| 0.3751695755129382| 2.357292648483867|10482793|
+--------------+---------+-------------------+------------------+--------+

In [23]:
#save two dataframe which contains delay information regarding to stop_id and transport_type
group_stop_id.write.parquet('/user/{0}/stop_delay.parquet'.format("zhou"))
#group_trip_id.write.mode("overwrite").parquet('/user/{0}/trip_id_delay.parquet'.format("zhou"))
group_transport_id.write.parquet('/user/{0}/transport_delay.parquet'.format("zhou"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
sc.stop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…